# Exogenous Processes

In models without exogenous processes the law of motion, the transition of states over periods, is deterministic. Given a state and a choice the state in the next period is certain.

Exogenous processes introduce a stochastic element to the law of motion. Given a state and a choice there exist multiple states in the next period which are valid successors. The transition to one of the successors is determined by a probability which depends on the characteristics of the state.

In [1]:
import pandas as pd
import numpy as np
import respy as rp

pd.set_option("display.max_rows", 100)

## A single exogenous process

In [19]:
params, options = rp.get_example_model("robinson_crusoe_basic", with_data=False)

In [20]:
params.loc[("nonpec_fishing", "sick"), "value"] = -2

In [21]:
params.loc[("exogenous_process_illness_sick", "probability"), "value"] = 0.1
params.loc[("exogenous_process_illness_healthy", "probability"), "value"] = 0.9

In [22]:
options["covariates"]["sick"] = "illness == 'sick'"

In [23]:
solve = rp.get_solve_func(params, options)

In [24]:
state_space = solve(params)

In [26]:
state_space.get_continuation_values(period=5)

{(0,): array([[11.9443976 , 10.63068531],
        [11.18086788,  9.93103142],
        [10.51774118,  9.32312399],
        [ 9.94525487,  8.79512137],
        [ 9.45037552,  8.33913529],
        [ 9.94525487,  8.79512137],
        [10.51774118,  9.32312399],
        [11.9443976 , 10.63068531],
        [11.18086788,  9.93103142],
        [12.82278203, 11.4327238 ]]), (1,): array([[ 9.61536203,  8.27815169],
        [ 9.07564118,  7.80039665],
        [ 8.62413497,  7.40001434],
        [ 8.24934272,  7.06447462],
        [ 7.93525973,  6.78474868],
        [ 8.24934272,  7.06447462],
        [ 8.62413497,  7.40001434],
        [ 9.61536203,  8.27815169],
        [ 9.07564118,  7.80039665],
        [10.25625295,  8.84665564]])}

In [9]:
simulate = rp.get_simulate_func(params, options)

In [10]:
df = simulate(params)

In [13]:
df["Prev_Illness"] = df.groupby("Identifier").Illness.shift()
pd.crosstab(df.Illness, df.Prev_Illness, normalize=True).to_numpy()

In [18]:
df["Prev_Weather"] = df.groupby("Identifier").Weather.shift()
pd.crosstab(df.Weather, df.Prev_Weather, normalize=True).to_numpy()

array([[0.63233333, 0.16177778],
       [0.16366667, 0.04222222]])

In [14]:
loglike = rp.get_crit_func(params, options, df)

In [15]:
loglike(params)

-14.078999941460754

In [16]:
asdasdasd

NameError: name 'asdasdasd' is not defined

## Exogenous process affecting availability of choices

In [ ]:
params.loc[("exogenous_process_weather_tropical_storm", "probability"), "value"] = 0.2
params.loc[("exogenous_process_weather_sunny", "probability"), "value"] = 0.8

In [ ]:
options["is_inadmissible"] = {
    "fishing": ["weather == 'tropical_storm'"]
}

In [ ]:
solve = rp.get_solve_func(params, options)

In [ ]:
state_space = solve(params)

In [ ]:
df = simulate(params)

In [ ]:
%debug

## References

> Behrman, J. R., Tincani, M. M., Todd, P. E., & Wolpin, K. I. (2016). [Teacher quality in public and private schools under a voucher system: The case of Chile.](https://scholarship.rice.edu/bitstream/handle/1911/94185/Teacher-Quality-Chile.pdf) *Journal of Labor Economics, 34(2)*, 319-362.